# VS Code와 Google Colab SSH 연동 가이드 (Cloudflared)

이 노트북은 VS Code에서 SSH를 통해 Google Colab의 GPU 리소스를 사용하는 방법을 설명합니다. cloudflared를 사용하여 안전하고 빠른 연결을 제공합니다.

## 1. 필요한 확장 프로그램 설치

VS Code에서 다음 확장 프로그램을 설치하세요:

1. Remote - SSH extension (ms-vscode-remote.remote-ssh)
   - VS Code에서 원격 SSH 연결을 위해 필요
   - 이미 설치되어 있다면 건너뛰기

2. GPU 설정
   - 런타임 > 런타임 유형 변경 > GPU 선택

## 2. SSH 연결 설정

1. cloudflared 설치 및 설정:
   - cloudflared 다운로드 및 설정
   - SSH 키 생성
   - 터널 설정

2. VS Code SSH 구성:
   - SSH 설정 파일 생성
   - 원격 연결 설정

In [ ]:
# 1. cloudflared 설치 및 설정
!pip install cloudflared

# 2. cloudflared 다운로드 및 실행 권한 설정
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# 3. SSH 설정
!mkdir -p ~/.ssh
!echo "PubkeyAcceptedKeyTypes=+ssh-rsa" > ~/.ssh/config
!chmod 600 ~/.ssh/config

# 4. SSH 키 생성
!ssh-keygen -t rsa -b 4096 -N '' -f ~/.ssh/id_rsa

# 5. SSH 공개 키 출력 (이를 복사해두세요)
print("\n=== 아래 SSH 공개키를 복사하여 저장해두세요 ===")
!cat ~/.ssh/id_rsa.pub

## 3. SSH 터널 설정

1. cloudflared 터널 시작:
   - 터널을 통해 안전한 연결 생성
   - 연결 주소 확인

2. VS Code에서 연결:
   - SSH 설정 파일에 호스트 추가
   - 원격 연결 시작

In [ ]:
# 1. SSH 서비스 시작
!service ssh start

# 2. cloudflared 터널 시작
get_ipython().system_raw('./cloudflared-linux-amd64 tunnel --url ssh://localhost:22 --logfile cloudflared.log --metrics localhost:45678 </dev/null &>/dev/null &')

# 3. 연결 주소 확인 (약 15초 대기 후)
import time
import re
import subprocess
time.sleep(15)

with open('cloudflared.log', 'r') as f:
    for line in f:
        if 'trycloudflare.com' in line:
            match = re.search(r'https://([^\.]+\.trycloudflare\.com)', line)
            if match:
                hostname = match.group(1)
                print("\n=== VS Code SSH 설정 ===")
                print("1. VS Code에서 Remote-SSH: Add New SSH Host를 선택")
                print("2. 다음 설정을 입력:")
                print(f"ssh root@{hostname}")
                print("\n3. 설정 파일 선택 후 연결")
                break

## 4. SSH 연결 및 개발 시작

SSH 연결이 성공적으로 설정되면:

1. VS Code에서:
   - 원격 터미널에서 코드 실행 가능
   - 파일 직접 수정 가능
   - Git 버전 관리 사용 가능

2. 주의사항:
   - Colab 세션이 종료되면 재연결 필요
   - 새로운 세션마다 새 SSH 키와 주소 사용
   - 중요한 파일은 주기적으로 백업

In [ ]:
# 1. SSH 서비스 상태 확인
!service ssh status

# 2. cloudflared 프로세스 확인
!ps aux | grep cloudflared

# 3. CUDA 가용성 확인
import torch
print(f"\nCUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 모델: {torch.cuda.get_device_name(0)}")
    print(f"GPU 개수: {torch.cuda.device_count()}")

print("\n=== 연결 테스트 완료 ===")
print("이제 VS Code에서 SSH를 통해 접속하여 개발을 시작할 수 있습니다.")